In [12]:
!pwd

/Users/vtadinad/Documents/PythonProjects/naga/scratch


In [13]:
import json 
import pprint 
pp = pprint.PrettyPrinter(indent=4)  
import glob 

In [ ]:
files = glob.glob("jsonsfiles/*.json")  
i = 0
for file in files[:]: 
    i += 1
    print(i, file) 
    f = open(file) 
    try:
        mj = json.load(f) 
        parse_json(mj,file)
    except:
        print("===========>error in ", file)

In [ ]:
def parse_json(j,f):
    if type(j) is not dict:
        print ("===============>",f, " does not have a job")
        return
    for k,v in j.items():
        if type(v) is dict:
            parse_json(v,f)
    if 'Type' in j:
        if j["Type"][:3] == "Job":            
            fname = "jobs/"+j["Type"].replace(":","-")+".json"
            print ("      found job in ",f,fname) 
            with open(fname, 'w') as out_file:
                out_file.write(json.dumps(j,indent=4))
files = glob.glob("jsonsfiles/*.json")  
i = 0
for file in files[:]: 
    i += 1
    print(i, file) 
    f = open(file) 
    mj = json.load(f) 
    parse_json(mj,file)

In [16]:
def gen_name(n):
    import re
    n = re.sub('([A-Z]{1})', r'_\1',n).lower()
    if n[0] == "_":
        n = n[1:]
    return n

def get_job_name(s):
    s =  s.split(":")[-1]
    return s
def json_to_job(j):
    job_type = j["Type"]
    job_name = get_job_name(job_type) + "Job"
    job_name = job_name.replace(" ","_")
    folder_name = job_type.replace(":","/").split("/")[:-1]
    folder_name = "../new" + "/".join(folder_name) 
    folder_name = folder_name.replace("Job","Jobs")
    file_name = folder_name + "/" + job_name + ".py"
    print(job_type, file_name, job_name)
    try:
        os.makedirs(folder_name)
        with open(folder_name+"/__init__.py","w") as f:
            f.write("")

    except FileExistsError:
        # directory already exists
        pass
    with open(file_name,"w") as f:
        params = {}
        f.write("from naga.jobs.base import BaseJob\n")
        f.write("\n")
        f.write("class {}(BaseJob):\n".format(job_name))
        f.write("\tdef __init__(self, folder, job_name, \n")
        for k in j.keys():
            if k not in ["Type","RunAs","Host"]:
                params[k] = gen_name(k)
                 f.write("\t\t\tif {} not None:},\n".format(params[k]))             
                f.write("\t\t\t\t{},\n".format(params[k]))
        f.write("\t\t\thost=None, run_as=None, description=None):\n")
        f.write("\t\tBaseJob.__init__(self, folder, job_name, description=description, host=host, run_as=run_as)\n")
        for k in params:
            f.write("\t\tself.{} = {}\n".format(params[k],params[k]))      
        
        f.write("\n")
        f.write("\tdef get_json(self):\n")
        f.write("\t\tjob_json = BaseJob.get_json(self)\n")
        f.write("\t\tjob_json['Type'] = '{}'\n".format(job_type))
        for k in params:
            f.write("\t\tjob_json['{}'] = self.{}\n".format(k,params[k]) )
        f.write("\t\treturn job_json\n")

 






files = glob.glob("jobs/*.json")  
for file in files[:]: 
    f = open(file) 
    mj = json.load(f) 
    json_to_job(mj)

IndentationError: unexpected indent (2207063639.py, line 37)

In [17]:
!pwd

/Users/vtadinad/Documents/PythonProjects/naga/scratch


In [25]:
import pandas as pd
def none_params():
    df = pd.read_csv("jobs/none_params.csv",sep=",",index_col=0)
    print(df.info())
    return df.to_dict()
none_params()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, 'EmbeddedQuery' to 'EmbeddedQuery'
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   col1    1 non-null      object
 1   col2    1 non-null      object
 2   col3    1 non-null      object
 3   col4    1 non-null      object
dtypes: object(4)
memory usage: 40.0+ bytes
None


{'col1': {"'EmbeddedQuery'": "'Autocommit'"},
 'col2': {"'EmbeddedQuery'": "'OutputExecutionLog'"},
 'col3': {"'EmbeddedQuery'": "'OutputSQLOutput'"},
 'col4': {"'EmbeddedQuery'": "'SQLOutputFormat'"}}

In [40]:
df = pd.read_csv("jobs/none_params.csv",sep=",",index_col=0)
df.info()
jt = list(df.index.values)
df.loc["EmbeddedQuery"].values.tolist()

<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, EmbeddedQuery to FileWatcher
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   col1    2 non-null      object
 1   col2    2 non-null      object
 2   col3    2 non-null      object
 3   col4    2 non-null      object
dtypes: object(4)
memory usage: 80.0+ bytes


['Autocommit', 'OutputExecutionLog', 'OutputSQLOutput', 'SQLOutputFormat']

Index(['EmbeddedQuery'], dtype='object', name='job')